### imports

In [225]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [226]:
%autoreload 2

In [227]:
from regex_to_NFA import NFA_CLASS,NFA
from NFA_to_DFA import DFA_CLASS,State,Edge
import json
from parser_classes import EPSILON

### Functions

In [228]:
def json_to_nfa(json_path: str) -> NFA:
    """
    This function is used to convert the JSON data to NFA
    params:
        json_path: str: Path to the JSON file
    return:
        NFA: NFA object
    """
    # read the JSON file
    with open(json_path, "r") as file:
        json_data = json.load(file)
    # create the NFA object
    nfa = NFA(None, None, [], [])
    # create the states
    states = []
    acceping_index = 0
    for state in json_data:
        if state == "startingState":
            continue
        if json_data[state]["isTerminatingState"] == True:
            acceping_index = int(state[1:])
        states.append(State(name=state))
    nfa.states = states
    # set the start state becuase the name of the starting states is the index of it inside the states array
    nfa.start = states[int(json_data["startingState"][1:])]
    # set the accept state
    nfa.accept = states[acceping_index]
    # create the transitions
    transitions = []
    for state in json_data:
        if state == "startingState":
            continue
        for transition in json_data[state]:
            if transition == "isTerminatingState":
                continue

            # check if the transition is epsilon or not
            if transition.startswith("epsilon"):
                transitions.append(
                    Edge(
                        states[int(state[1:])],
                        states[int(json_data[state][transition][1:])],
                        {EPSILON},
                    )
                )
            else:
                transitions.append(
                    Edge(
                        states[int(state[1:])],
                        states[int(json_data[state][transition][1:])],
                        {transition},
                    )
                )
    nfa.transitions = transitions
    return nfa

### Regex to NFA

In [229]:
# regex = "[A-Za-z]+[0-9]*"
# regex = "ab*c+de?(f|g|h)|mr|n|[p-qs0-9]"
# regex = "ab"
regex = "(A|B)+"

In [230]:
nfa = NFA_CLASS(regex)

Visualization of the NFA is saved in ./nfa.gv


### NFA to DFA

In [231]:
# deserialize the json file to get the nfa
deserialized_nfa = json_to_nfa("./nfa.json")

In [232]:
# getting the inputs from the tansitions in the nfa
inputs = []
for transition in deserialized_nfa.transitions:
    temp_in = list(transition.characters)[0]
    if temp_in not in inputs and temp_in != EPSILON:
        inputs.append(temp_in)
inputs

['A', 'B']

In [233]:
dfa = DFA_CLASS()

In [234]:
dfa.subset_construction(deserialized_nfa, inputs)

Current State:  {State(name='S2'), State(name='S6'), State(name='S4'), State(name='S0')}
Input: A, Move: {State(name='S2'), State(name='S4'), State(name='S0'), State(name='S1'), State(name='S7'), State(name='S5'), State(name='S6')}
Input: B, Move: {State(name='S3'), State(name='S2'), State(name='S4'), State(name='S0'), State(name='S7'), State(name='S5'), State(name='S6')}
Current State:  {State(name='S2'), State(name='S4'), State(name='S0'), State(name='S1'), State(name='S7'), State(name='S5'), State(name='S6')}
Input: A, Move: {State(name='S2'), State(name='S4'), State(name='S0'), State(name='S1'), State(name='S7'), State(name='S5'), State(name='S6')}
Input: B, Move: {State(name='S3'), State(name='S2'), State(name='S4'), State(name='S0'), State(name='S7'), State(name='S5'), State(name='S6')}
Current State:  {State(name='S3'), State(name='S2'), State(name='S4'), State(name='S0'), State(name='S7'), State(name='S5'), State(name='S6')}
Input: A, Move: {State(name='S2'), State(name='S4'), 

In [235]:
dfa.print_dfa()

Start State:  S2,S6,S4,S0
Accepting States:  ['S2,S4,S0,S1,S7,S5,S6', 'S3,S2,S4,S0,S7,S5,S6']
States:  ['S2,S6,S4,S0', 'S2,S4,S0,S1,S7,S5,S6', 'S3,S2,S4,S0,S7,S5,S6']
Transitions: 
From: S2,S6,S4,S0, To: S2,S4,S0,S1,S7,S5,S6, Characters: {'A'}
From: S2,S6,S4,S0, To: S3,S2,S4,S0,S7,S5,S6, Characters: {'B'}
From: S2,S4,S0,S1,S7,S5,S6, To: S2,S4,S0,S1,S7,S5,S6, Characters: {'A'}
From: S2,S4,S0,S1,S7,S5,S6, To: S3,S2,S4,S0,S7,S5,S6, Characters: {'B'}
From: S3,S2,S4,S0,S7,S5,S6, To: S2,S4,S0,S1,S7,S5,S6, Characters: {'A'}
From: S3,S2,S4,S0,S7,S5,S6, To: S3,S2,S4,S0,S7,S5,S6, Characters: {'B'}


In [236]:
dfa.rename_dfa_states()
dfa.print_dfa()

Start State:  S0
Accepting States:  ['S1', 'S2']
States:  ['S0', 'S1', 'S2']
Transitions: 
From: S0, To: S1, Characters: {'A'}
From: S0, To: S2, Characters: {'B'}
From: S1, To: S1, Characters: {'A'}
From: S1, To: S2, Characters: {'B'}
From: S2, To: S1, Characters: {'A'}
From: S2, To: S2, Characters: {'B'}


In [237]:
dfa.dfa_to_json("dfa.json")

In [238]:
dfa.visualize_dfa()

Visualization of the DFA is saved in ./dfa.gv


### Minimizing DFA

In [ ]:
minimized_dfa = dfa.minimize_dfa()

In [ ]:
# create a new object of DFA_CLASS to use the minimized dfa
minimized_dfa_obj = DFA_CLASS(minimized_dfa,inputs)

In [ ]:
# print the minimized dfa
minimized_dfa_obj.print_dfa()

In [ ]:
minimized_dfa_obj.dfa_to_json("minimized_dfa.json")

In [ ]:
minimized_dfa_obj.visualize_dfa()